In [15]:
import pandas as pd
import numpy as np
import requests
import asyncio
import aiohttp
import os
from dotenv import load_dotenv
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [2]:
load_dotenv()

True

In [3]:
API_KEY = os.getenv("API_KEY")

In [4]:
API_KEY

'e9043dc6796e3ce6b7607e6d087f66af'

In [5]:
def analysis_time_series():
    pass

In [6]:
# Реальные средние температуры (примерные данные) для городов по сезонам
seasonal_temperatures = {
    "New York": {"winter": 0, "spring": 10, "summer": 25, "autumn": 15},
    "London": {"winter": 5, "spring": 11, "summer": 18, "autumn": 12},
    "Paris": {"winter": 4, "spring": 12, "summer": 20, "autumn": 13},
    "Tokyo": {"winter": 6, "spring": 15, "summer": 27, "autumn": 18},
    "Moscow": {"winter": -10, "spring": 5, "summer": 18, "autumn": 8},
    "Sydney": {"winter": 12, "spring": 18, "summer": 25, "autumn": 20},
    "Berlin": {"winter": 0, "spring": 10, "summer": 20, "autumn": 11},
    "Beijing": {"winter": -2, "spring": 13, "summer": 27, "autumn": 16},
    "Rio de Janeiro": {"winter": 20, "spring": 25, "summer": 30, "autumn": 25},
    "Dubai": {"winter": 20, "spring": 30, "summer": 40, "autumn": 30},
    "Los Angeles": {"winter": 15, "spring": 18, "summer": 25, "autumn": 20},
    "Singapore": {"winter": 27, "spring": 28, "summer": 28, "autumn": 27},
    "Mumbai": {"winter": 25, "spring": 30, "summer": 35, "autumn": 30},
    "Cairo": {"winter": 15, "spring": 25, "summer": 35, "autumn": 25},
    "Mexico City": {"winter": 12, "spring": 18, "summer": 20, "autumn": 15},
}

# Сопоставление месяцев с сезонами
month_to_season = {12: "winter", 1: "winter", 2: "winter",
                   3: "spring", 4: "spring", 5: "spring",
                   6: "summer", 7: "summer", 8: "summer",
                   9: "autumn", 10: "autumn", 11: "autumn"}

# Генерация данных о температуре
def generate_realistic_temperature_data(cities, num_years=15):
    dates = pd.date_range(start="2010-01-01", periods=365 * num_years, freq="D")
    data = []

    for city in cities:
        for date in dates:
            season = month_to_season[date.month]
            mean_temp = seasonal_temperatures[city][season]
            # Добавляем случайное отклонение
            temperature = np.random.normal(loc=mean_temp, scale=5)
            data.append({"city": city, "timestamp": date, "temperature": temperature})

    df = pd.DataFrame(data)
    df['season'] = df['timestamp'].dt.month.map(lambda x: month_to_season[x])
    return df

# Генерация данных
data = generate_realistic_temperature_data(list(seasonal_temperatures.keys()))
data.to_csv('temperature_data.csv', index=False)

In [16]:
data = pd.read_csv('temperature_data.csv')

In [17]:
data

,city,timestamp,temperature,season
0,New York,2010-01-01,1.728352,winter
1,New York,2010-01-02,0.509519,winter
2,New York,2010-01-03,7.409572,winter
3,New York,2010-01-04,-0.044949,winter
4,New York,2010-01-05,-8.751654,winter
...,...,...,...,...
82120,Mexico City,2024-12-23,2.102406,winter
82121,Mexico City,2024-12-24,15.099156,winter
82122,Mexico City,2024-12-25,14.641806,winter
82123,Mexico City,2024-12-26,9.278237,winter


In [7]:
def get_current_weather_for_city(city: str, API_KEY: str):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return (city, data)
    else:
        return None

In [8]:
get_current_weather_for_city("Moscow", API_KEY)

('Moscow',
 {'coord': {'lon': 37.6156, 'lat': 55.7522},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01n'}],
  'base': 'stations',
  'main': {'temp': -7.51,
   'feels_like': -13.55,
   'temp_min': -8.71,
   'temp_max': -7.41,
   'pressure': 1018,
   'humidity': 63,
   'sea_level': 1018,
   'grnd_level': 997},
  'visibility': 10000,
  'wind': {'speed': 4.05, 'deg': 316, 'gust': 8.38},
  'clouds': {'all': 6},
  'dt': 1765732417,
  'sys': {'type': 2,
   'id': 2094500,
   'country': 'RU',
   'sunrise': 1765691538,
   'sunset': 1765716973},
  'timezone': 10800,
  'id': 524901,
  'name': 'Moscow',
  'cod': 200})

In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 55.45,
	"longitude": 37.37,
	"start_date": "2025-01-01",
	"end_date": "2025-12-14",
	"hourly": "temperature_2m",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Coordinates: 55.430580139160156°N 37.379032135009766°E
Elevation: 183.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                           date  temperature_2m
0    2025-01-01 00:00:00+00:00          -2.778
1    2025-01-01 01:00:00+00:00          -4.078
2    2025-01-01 02:00:00+00:00          -4.578
3    2025-01-01 03:00:00+00:00          -5.178
4    2025-01-01 04:00:00+00:00          -5.728
...                        ...             ...
8347 2025-12-14 19:00:00+00:00         -10.328
8348 2025-12-14 20:00:00+00:00         -10.478
8349 2025-12-14 21:00:00+00:00         -10.578
8350 2025-12-14 22:00:00+00:00         -10.678
8351 2025-12-14 23:00:00+00:00         -10.728

[8352 rows x 2 columns]
